# Multimodal deep learning 


It employs the spectral and tabular deep learning models as modules, which will be trained together to inform the grain yield prediction.

In [ ]:
# Import libraries
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision.all import *
import fastai
from fastai.tabular.all import *
from fastai.data.load import _FakeLoader, _loaders
import torch
from ipywidgets import IntProgress
from glob import glob

import random
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import os

import fastcore

# Custom functions
from msi_utils_Multimodal import *
from fold_utils_Multimodal import * 
from multimodal_utils import *
from multimodal_model import *

In [ ]:
device = torch.device('hip:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
print(torch.cuda.is_available())

In [ ]:
print("fastai version:", fastai.__version__)
print("fastcore version:", fastcore.__version__)
print("torch version:", torch.__version__)

## Reference tables

In [ ]:
#Training/Val Set
path = Path('/path/train_images')
df_train_val = pd.read_csv('/path/Train_Val.csv')


In [ ]:
#Holdout set
df_test = pd.read_csv('/path/Holdout.csv')
path_test = Path('/path/holdout_images')

In [ ]:
# Random splitter function from fastai
splitter = RandomSplitter(valid_pct=0.3, seed=42)
splits = splitter(range_of(df_train_val))
splits

# Tab Dataloaders

In [ ]:
procs = [Categorify, Normalize, FillMissing]
cont_vars = df_train_val.columns[21:].tolist()
additional_cont_vars = ['JulianPlantDatePerYear', 'Year', 'DTA', 'DTS', 'Moist', 'Population', 'Range', 'Row']
cont_names =  cont_vars + additional_cont_vars 
cat_names = ['Pedigree1', 'Pedigree2', 'Stock', 'Test']

to = TabularPandas(df_train_val,
                   procs,
                   cat_names=cat_names,
                   cont_names=cont_names,
                   y_names='Yield',
                   y_block=RegressionBlock(),
                   splits=splits)

tab_dl = to.dataloaders(bs=64)

In [ ]:
tab_dl.show_batch()

# Spectral Dataloaders

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, RegressionBlock),
            get_items=get_image_files_from_df,
            get_y=get_y,
            splitter=splitter,
            item_tfms=[FlipItem, Resize(360, None)],
            batch_tfms=[Normalize])


In [ ]:
msi_dls = dblock.dataloaders(path, bs=64)

In [ ]:
msi_dls.show_batch()

# Mixed Dataloader

In [ ]:
# Check that the tabular dataset is aligned with the spectral dataset
mixed_dl = MixedDL(tab_dl[0], msi_dls[0])
# These should show the same ids
msi_dls[0].get_idxs()[:10]

In [ ]:
tab_dl[0].get_idxs()[:10]

In [ ]:
# Now mix the tabular and spectral datasets to create the multimodal input
train_mixed_dl = MixedDL(tab_dl[0], msi_dls[0])
valid_mixed_dl = MixedDL(tab_dl[1], msi_dls[1])
mixed_dls = DataLoaders(train_mixed_dl, valid_mixed_dl).cuda()

In [ ]:
mixed_dls.show_batch()

# Fusion at feature level - Training modules from scratch

## Training & Validation

In [ ]:
# Mixed model variables
# Set weights for each loss
tab_w, vis_w, tv_w = 0.42, 0.34, 0.24 #Modify depending on Optuna

# Initialise Loss
gb_loss = GradientBlending(tab_weight=tab_w, visual_weight=vis_w, tab_vis_weight=tv_w, loss_scale=1.0)

In [ ]:
# METRICS
metrics = [t_rmse, v_rmse, tv_rmse, weighted_RMSEp]
csvlogger = CSVLogger(f'/path/metrics.csv', append=True)
cbs = [csvlogger]    

In [ ]:
# Modules
config = tabular_config(ps=0.5, embed_p=0.5)
learn_tab = tabular_learner(tab_dl,
                            config=config,
                            layers=[200,100],
                            metrics=[rmse, R2Score()],
                            opt_func=ranger,
                            y_range=[0,20],
                            wd=1.425482107813348e-06)

learn_tab.fit_one_cycle(1, lr_max=0.00018479913871295546)

In [ ]:
model_msi = models.densenet121(pretrained=True)

# Modify the architecture to have 1 output classes
num_classes = 1
model_msi.classifier = nn.Linear(model_msi.classifier.in_features, num_classes)

# Add this line after creating the model architecture
learn_rgb = Learner(msi_dls,
            model_msi,
            opt_func=RAdam,
            loss_func=root_mean_squared_error,  
            metrics=[rmse, R2Score()])

learn_rgb.fit(1, lr=0.0001289, wd=0.000137)

In [ ]:
multi_model = TabVis(learn_tab.model, learn_rgb.model)
multi_learn = Learner(mixed_dls, multi_model, gb_loss, cbs=cbs, metrics=metrics)

In [ ]:
# Disable Fastai progress bar
with multi_learn.no_bar() and multi_learn.no_logging():
    multi_learn.fit_one_cycle(100, lr_max=0.000183557)

In [ ]:
multi_learn.save('/path/Multimodal_FromScratch') #SAVE MODEL FOR FUTURE EVALUATION.


In [ ]:
pn = msi_dls.valid_ds.items
images_id = []

In [ ]:
for i in range(len(pn)):
    path = Path(pn[i])  # Convert the file path to a Path object
    name = path.stem
    images_id.append(name)

In [ ]:
preds,targs = multi_learn.get_preds(dl=valid_mixed_dl)
pred_mixed_df = pd.DataFrame()
tab_pred = preds[0].flatten()
vis_pred = preds[1].flatten()
mixed_pred = preds[2].flatten()

pred_mixed_df['items'] = images_id
pred_mixed_df['items'] = pred_mixed_df['items'].str.replace('id_', '')
pred_mixed_df['tab_pred'] = tab_pred
pred_mixed_df['msi_pred'] = vis_pred
pred_mixed_df['mixed_pred'] = mixed_pred
pred_mixed_df['target_yield'] = targs


In [ ]:
pred_mixed_df.to_csv('/path/Preds.csv')


## Holdout Evaluation

In [ ]:
#SKIP IF MULTI_LEARN HAS ALREADY BEEN INITIALISED/LOADED.

multi_learn.load('/path/Multimodal_FromScratch')

In [ ]:
test_msi_dls = dblock.dataloaders(path_test, shuffle=False)
learn_rgb.dls.loaders.append(msi_dls.test_dl(test_msi_dls.items, with_labels=True, shuffle=False))

In [ ]:
len(test_msi_dls.items)

In [ ]:
# 1st half -
# Find the order of samples in the MSI test DL

fnames_MSIorder =[]
for fname in test_msi_dls.items:
    fname = str(fname)
    fname = fname.split(sep='/')[-1]
    fname = fname.replace('.npy', '')
    fname = fname[3:18]
    fnames_MSIorder.append(fname)

In [ ]:
# Reorder the df_test to reflect this order
df_test1 = df_test.set_index('Replicate').reindex(fnames_MSIorder).reset_index()

In [ ]:
learn_tab.dls.loaders.append(tab_dl.test_dl(df_test1, with_labels=True, shuffle=False))

In [ ]:
test_mixed_dl = MixedDL(learn_tab.dls[2], learn_rgb.dls[2])
test_mixed_dl.show_batch()

In [ ]:
preds,targs = multi_learn.get_preds(dl=test_mixed_dl)
tab_pred = preds[0].flatten()
vis_pred = preds[1].flatten()
mixed_pred = preds[2].flatten()

mixed_results = df_test1.copy()
mixed_results['tab_pred'] = tab_pred
mixed_results['msi_pred'] = vis_pred
mixed_results['mixed_pred'] = mixed_pred

len(mixed_results)

In [ ]:
# 2nd half -
# Find the order of samples in the MSI test DL

fnames_MSIorder =[]
for fname in test_msi_dls[1].items:
    fname = str(fname)
    fname = fname.split(sep='/')[-1]
    fname = fname.replace('.npy', '')
    fname = fname[3:18]
    fnames_MSIorder.append(fname)
    
# Reorder the df_test to reflect this order
df_test2 = df_test.set_index('Replicate').reindex(fnames_MSIorder).reset_index()
learn_tab.dls.loaders.append(tab_dl.test_dl(df_test2, with_labels=True, shuffle=False))
learn_rgb.dls.loaders.append(msi_dls.test_dl(test_msi_dls[1].items, with_labels=True, shuffle=False))

In [ ]:
test_mixed_dl = MixedDL(learn_tab.dls[3], learn_rgb.dls[3])
test_mixed_dl.show_batch()

In [ ]:
preds,targs = multi_learn.get_preds(dl=test_mixed_dl)
tab_pred = preds[0].flatten()
vis_pred = preds[1].flatten()
mixed_pred = preds[2].flatten()

mixed_results2 = df_test2.copy()
mixed_results2['tab_pred'] = tab_pred
mixed_results2['msi_pred'] = vis_pred
mixed_results2['mixed_pred'] = mixed_pred

len(mixed_results2)

In [ ]:
ff_GB_results = mixed_results
ff_GB_results = mixed_results.append(mixed_results2)
ff_GB_results

In [ ]:
ff_GB_results.to_csv('/path/preds.csv')

# Fusion at feature level - Pretrained modules 

## Training and Validation

In [ ]:
# Mixed model variables
# Set weights for each loss
tab_w, vis_w, tv_w = 0.42, 0.34, 0.24 #Modify depending on Optuna

# Initialise Loss
gb_loss = GradientBlending(tab_weight=tab_w, visual_weight=vis_w, tab_vis_weight=tv_w, loss_scale=1.0)

# METRICS
metrics = [t_rmse, v_rmse, tv_rmse, weighted_RMSEp]
csvlogger = CSVLogger(f'/model/Metrics.csv', append=True)
cbs = [csvlogger]    

# Modules
config = tabular_config(ps=0.5, embed_p=0.5)
learn_tab = tabular_learner(tab_dl,
                            config=config,
                            layers=[200,100],
                            metrics=[rmse, R2Score()],
                            opt_func=ranger,
                            y_range=[0,20],
                            wd=1.425482107813348e-06)

learn_tab.load('/path/Tabular_Model')

learn_tab.fit_one_cycle(1, lr_max=0.00018479913871295546)

model_msi = models.densenet121(pretrained=True)

# Modify the architecture to have 1 output classes
num_classes = 1
model_msi.classifier = nn.Linear(model_msi.classifier.in_features, num_classes)

# Add this line after creating the model architecture
learn_rgb = Learner(msi_dls,
            model_msi,
            opt_func=RAdam,
            loss_func=root_mean_squared_error,  
            metrics=[rmse, R2Score()])

learn_rgb.load('/path/Image_Model')

learn_rgb.fit(1, lr=0.0001289, wd=0.000137)

multi_model = TabVis(learn_tab.model, learn_rgb.model)
multi_learn = Learner(mixed_dls, multi_model, gb_loss, cbs=cbs, metrics=metrics)
    
    
# Disable Fastai progress bar
with multi_learn.no_bar() and multi_learn.no_logging():
     multi_learn.fit_one_cycle(100, lr_max=0.000183557)
    

In [ ]:
multi_learn.save('/path/Multimodal_Pretrained') #SAVE MODEL FOR FUTURE EVALUATION.

In [ ]:
pn = msi_dls.valid_ds.items
images_id = []

for i in range(len(pn)):
    path = Path(pn[i])  # Convert the file path to a Path object
    name = path.stem
    images_id.append(name)
    
preds,targs = multi_learn.get_preds(dl=valid_mixed_dl)
pred_mixed_df = pd.DataFrame()
tab_pred = preds[0].flatten()
vis_pred = preds[1].flatten()
mixed_pred = preds[2].flatten()

pred_mixed_df['items'] = images_id
pred_mixed_df['items'] = pred_mixed_df['items'].str.replace('id_', '')
pred_mixed_df['tab_pred'] = tab_pred
pred_mixed_df['msi_pred'] = vis_pred
pred_mixed_df['mixed_pred'] = mixed_pred
pred_mixed_df['target_yield'] = targs

In [ ]:
pred_mixed_df.to_csv('/path/Preds.csv')

## Holdout Evaluation

In [ ]:
#SKIP IF MULTI_LEARN HAS ALREADY BEEN INITIALISED/LOADED.

multi_learn.load('/path/Multimodal_Pretrained')

In [ ]:
test_msi_dls = dblock.dataloaders(path_test, shuffle=False)
learn_rgb.dls.loaders.append(msi_dls.test_dl(test_msi_dls.items, with_labels=True, shuffle=False))

In [ ]:
# 1st half -
# Find the order of samples in the MSI test DL

fnames_MSIorder =[]
for fname in test_msi_dls.items:
    fname = str(fname)
    fname = fname.split(sep='/')[-1]
    fname = fname.replace('.npy', '')
    fname = fname[3:18]
    fnames_MSIorder.append(fname)

In [ ]:
# Reorder the df_test to reflect this order
df_test1 = df_test.set_index('Replicate').reindex(fnames_MSIorder).reset_index()

In [ ]:
learn_tab.dls.loaders.append(tab_dl.test_dl(df_test1, with_labels=True, shuffle=False))

In [ ]:
test_mixed_dl = MixedDL(learn_tab.dls[2], learn_rgb.dls[2])
test_mixed_dl.show_batch()

In [ ]:
preds,targs = multi_learn.get_preds(dl=test_mixed_dl)
tab_pred = preds[0].flatten()
vis_pred = preds[1].flatten()
mixed_pred = preds[2].flatten()

mixed_results = df_test1.copy()
mixed_results['tab_pred'] = tab_pred
mixed_results['msi_pred'] = vis_pred
mixed_results['mixed_pred'] = mixed_pred

len(mixed_results)

In [ ]:
# 2nd half -
# Find the order of samples in the MSI test DL

fnames_MSIorder =[]
for fname in test_msi_dls[1].items:
    fname = str(fname)
    fname = fname.split(sep='/')[-1]
    fname = fname.replace('.npy', '')
    fname = fname[3:18]
    fnames_MSIorder.append(fname)
    
# Reorder the df_test to reflect this order
df_test2 = df_test.set_index('Replicate').reindex(fnames_MSIorder).reset_index()
learn_tab.dls.loaders.append(tab_dl.test_dl(df_test2, with_labels=True, shuffle=False))
learn_rgb.dls.loaders.append(msi_dls.test_dl(test_msi_dls[1].items, with_labels=True, shuffle=False))

In [ ]:
test_mixed_dl = MixedDL(learn_tab.dls[3], learn_rgb.dls[3])
test_mixed_dl.show_batch()

In [ ]:
preds,targs = multi_learn.get_preds(dl=test_mixed_dl)
tab_pred = preds[0].flatten()
vis_pred = preds[1].flatten()
mixed_pred = preds[2].flatten()

mixed_results2 = df_test2.copy()
mixed_results2['tab_pred'] = tab_pred
mixed_results2['msi_pred'] = vis_pred
mixed_results2['mixed_pred'] = mixed_pred

len(mixed_results2)

In [ ]:
ff_GB_results = mixed_results
ff_GB_results = mixed_results.append(mixed_results2)
ff_GB_results

In [ ]:
ff_GB_results.to_csv('/path/Preds.csv')